In [1]:
import os

os.environ["CAPSTONE_OPENAI_API"] = ""


In [2]:
from agents.technical_agent import TechnicalAgent
import torch, json, os

# 1. 에이전트 생성
agent = TechnicalAgent(ticker="NVDA")


In [3]:
agent

TechnicalAgent(
  (lstm1): LSTM(13, 64, batch_first=True)
  (lstm2): LSTM(64, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (drop): Dropout(p=0.18778570103014075, inplace=False)
  (loss_fn): HuberLoss()
)

In [4]:
# 2. 데이터 수집 및 지표 계산(5개년) → yfinance에서 5년 데이터 다시 받음 + 13개 TECH 지표 생성
X = agent.searcher(rebuild=True)

⚙️ NVDA TechnicalAgent dataset not found. Building new dataset...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (982, 55, 13), y_seq: (982, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (982 samples, 13 features)
[FundamentalAgent] X_seq: (1222, 14, 13), y_seq: (1222, 1)
✅ NVDA FundamentalAgent dataset saved to CSV (1222 samples, 13 features)
[SentimentalAgent] X_seq: (1222, 14, 8), y_seq: (1222, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1222 samples, 8 features)


C:\Users\ska69\capstone_project\agents\base_agent.py:241: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="5y", interval="1d") # 아연수정
[*********************100%***********************]  1 of 1 completed


■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


In [5]:
X.shape

torch.Size([1, 55, 13])

In [6]:
# 3. 모델 학습
# 5년 데이터 기반 LSTM 학습 + 스케일러 저장 + 가중치 저장
agent.pretrain()

[20:41:49] Pretraining TechnicalAgent
  Epoch 005 | Loss: 0.396272
  Epoch 010 | Loss: 0.397418
  Epoch 015 | Loss: 0.386288
  Epoch 020 | Loss: 0.393506
  Epoch 025 | Loss: 0.415031
  Epoch 030 | Loss: 0.384957
  Epoch 035 | Loss: 0.380922
  Epoch 040 | Loss: 0.396026
  Epoch 045 | Loss: 0.388294
 TechnicalAgent 모델 학습 및 저장 완료: models\NVDA_TechnicalAgent.pt


In [8]:
# 4. 최신 윈도우 확보 → 예측 → 설명 생성 → 저장
from config.agents import dir_info

# 최신 윈도우
X_last = agent.searcher(agent.ticker)      # (1,T,F)
dates  = getattr(agent.stockdata, f"{agent.agent_id}_dates", [])


C:\Users\ska69\capstone_project\agents\base_agent.py:241: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="5y", interval="1d") # 아연수정
[*********************100%***********************]  1 of 1 completed


■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


In [9]:
print(X_last.shape)
print(dates[:5])

torch.Size([1, 55, 13])
['2025-08-13', '2025-08-14', '2025-08-15', '2025-08-18', '2025-08-19']


In [10]:
from agents.base_agent import Target
print(Target.__annotations__)


{'next_close': 'float', 'uncertainty': 'float | None', 'confidence': 'float | None', 'idea': 'dict | None'}


In [11]:
# 5) 예측 + 불확실성
tgt = agent.predict(X_last)            # Target(next_close, uncertainty, confidence, idea)

tgt

Target(next_close=203.4229, uncertainty=0.0029, confidence=0.9971, idea={'per_time': [{'date': '2025-08-13', 'sum_abs': [0.011881801388104202, 0.013080803597887833, 0.014525337167489752, 0.015542533930468238, 0.01592686444178335, 0.017215316724635908, 0.01752472987487981, 0.017392836100755336, 0.017104091499574946, 0.017104452275947965, 0.01759629640498385, 0.018216214435013277, 0.019646915837541308, 0.02106089247137216, 0.02070542724245353, 0.020432912361182182, 0.021310235912632765, 0.021533465375007253, 0.021121280077376976, 0.019985786275561992, 0.01929349585183047, 0.019594479476932505, 0.019790438761156805, 0.019923701456247382, 0.02117832781190862, 0.019897429650893965, 0.019127463458912336, 0.01807955811151702, 0.018413906798685105, 0.019503957457438888, 0.019858795687820955, 0.019259740480989114, 0.019194001933042715, 0.01803856274693633, 0.017270826344132902, 0.017383596122928772, 0.01767811514517808, 0.01862636322676537, 0.01839736394886924, 0.017637266580222376, 0.016927567

In [12]:
# 6) 저장
from agents.base_agent import save_explain_json, load_explain_json

save_explain_json(agent.ticker, agent.agent_id, tgt)

'models/explain\\NVDA_TechnicalAgent_explain.json'

In [13]:
# 6. LLM 메시지 구성 (system, user)
sys_text, user_text = agent._build_messages_opinion(agent.stockdata, tgt)
print(sys_text[:400], "\n---\n", user_text[:700])

C:\Users\ska69\capstone_project\agents\base_agent.py:241: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="5y", interval="1d") # 아연수정
[*********************100%***********************]  1 of 1 completed


■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
너는 '기술적 분석 전문가'다. 입력 컨텍스트(ctx, JSON)를 기반으로
            다음날 종가 예측이 타당한 이유만 한국어로 작성하라.
            출력은 반드시 JSON 객체 {{"reason": string}} 하나만.
            예측 재계산 및 새로운 수치 생성 금지(단, 제공된 수치의 곱을 이용한 영향 점수 표기는 허용).
            개발자 용어 사용 금지: attention/어텐션, SHAP/샤프, feature 중요도/피처 중요도, Grad×Input, Occlusion, 게이트, 신뢰도, 베타, 알파 등.
            항상 사용자 중심 표현을 사용하고, ctx에 주어진 수치만 활용하라. 
---
 ctx(JSON):
            {"ticker": "NVDA", "last_price": 202.49, "next_close": 203.4229, "uncertainty": 0.0029, "confidence": 0.9971, "sigma": 0.0029, "beta": 0.9971, "window_size": 55, "idea": {"per_time": [{"date": "2025-08-13", "sum_abs": [0.011881801388104202, 0.013080803597887833, 0.014525337167489752, 0.015542533930468238, 0.01592686444178335, 0.017215316724635908, 0.01752472987487981, 0.017392836100755336, 0.017104091499574946, 0.017104452275947965, 0.01759629640498385, 0.018216214435013277, 0.019646915837541308, 0.02106089247137216, 0.02070542724245353, 0.0204329123

In [14]:
# 7. LLM 호출 및 응답 확인
op = agent.reviewer_draft(agent.stockdata, tgt)
print(op.reason)

C:\Users\ska69\capstone_project\agents\base_agent.py:241: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="5y", interval="1d") # 아연수정
[*********************100%***********************]  1 of 1 completed


■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
1) obv/2025-08-14 (0.4102×0.0218=0.0089): OBV는 거래량과 가격 변동을 결합해 매수세를 보여주며, 상승 신호로 해석됩니다. 2) vol_20d/2025-08-14 (0.4041×0.0218=0.0088): 20일 거래량은 시장 참여도와 거래 활발함을 나타내어 추세 강도를 판단하는 데 중요합니다. 3) ret_3d/2025-08-14 (0.3784×0.0218=0.0083): 3일 수익률은 단기 가격 변동성을 반영해 단기 추세를 파악하는 데 도움을 줍니다. 4) log_ret_lag1/2025-08-14 (0.3693×0.0218=0.0081): 전일 로그 수익률은 최근 가격 움직임의 연속성을 보여줍니다. 2025-08-13~2025-08-14 기간은 거래량과 수익률 지표들이 모두 높은 기여도를 보이며, 강한 매수세와 활발한 거래가 지속된 상방 추세를 나타냅니다. 영향 점수 순위에서 OBV와 20일 거래량이 거의 동등한 높은 점수를 보여, 거래량 기반 매수세가 가격 상승을 견인했음을 알 수 있습니다. 3일 수익률과 전일 로그 수익률도 높은 점수를 기록해 단기 가격 상승 모멘텀이 강했음을 뒷받침합니다. 반면, 되돌림이나 과열 신호로 해석될 수 있는 지표들은 상대적으로 낮은 영향 점수(예: 0.0003 이하)로 나타나, 상승 흐름에 큰 제약을 주지 않았습니다. 종합하면, 거래량과 가격 모멘텀 지표들이 일관되게 긍정적 신호를 보내고 있어, 다음날 종가 예측이 상승 방향으로 안정적임을 사용자 입장에서 신뢰할 수 있습니다.
